In [ ]:
import numpy as np
import pandas as pd
import sys
import re
import lightgbm as lgb

In [ ]:
data_path='F:\\study\\ml\\DataSet\\Drug_Molecule\\'
df_protein_train=pd.read_csv('%sdf_protein_train.csv' % (data_path))
df_protein_test=pd.read_csv('%sdf_protein_test.csv' % (data_path))
df_molecule=pd.read_csv('%sdf_molecule.csv'% (data_path))
df_affinity_train=pd.read_csv('%sdf_affinity_train.csv'% (data_path))
df_affinity_test=pd.read_csv('%sdf_affinity_test.csv'% (data_path))

In [ ]:
df_affinity_test['Ki']=-11
data=pd.concat([df_affinity_train,df_affinity_test])

In [ ]:
protein_concat = pd.concat([df_protein_train,df_protein_test])

In [ ]:
df_molecule.head()

In [ ]:
feat=df_molecule.Fingerprint.apply(lambda x:x.split(',')).tolist()
## Series to list

In [ ]:
feat[1]

In [ ]:
feat=pd.DataFrame(feat)

In [ ]:
feat.columns=['Fingerprint_{}'.format(i) for i in range(feat.shape[1])]
feat.astype('int')

In [ ]:
feat['Molecule_ID']=df_molecule.Molecule_ID

In [ ]:
df_molecule_new=df_molecule.merge(feat,on='Molecule_ID',how='left')
del df_molecule_new['Fingerprint']

In [ ]:
df_molecule_new.head()

In [ ]:
protein_concat.Sequence[0]

In [ ]:
protein_concat.head()

In [ ]:
import re

In [ ]:
texts=[[word for word in re.findall(r'.{3}',document)]
      for document in list(protein_concat.Sequence)]

In [ ]:
texts[1]

In [ ]:
texts

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model=Word2Vec(texts,window=8,min_count=2,size=256,hs=1,negative=4,sg=1,sample=0.01,workers=4)

In [ ]:
model.wv.vocab

In [ ]:
model.wv.vectors

In [ ]:
a=[protein_concat.Protein_ID]
type(a[0])

In [ ]:
b=list(protein_concat.Protein_ID)

In [ ]:
b[0]

In [ ]:
type(b[0])

In [ ]:
featw=pd.DataFrame()
aa=list(protein_concat.Protein_ID)
result1=[]
for i in range(len(texts)):
    result2=[]
    for w in range(len(texts[i])):
        result2.append(aa[i])
    result1.extend(result2)
featw['Protein_ID']=result1
    

In [ ]:
featw.head()

In [ ]:
featw=pd.DataFrame()
aa=list(protein_concat.Protein_ID)
result1=[]
for i in range(len(texts)):
    result2=[]
    for w in range(len(texts[i])):
        result2.append(aa[i])
    result1.extend(result2)
featw['Protein_ID']=result1

In [ ]:
result1=[]
for i in range(len(texts)):
    result2=[]
    for w in range(len(texts[i])):
        result2.append(texts[i][w])
    result1.extend(result2)
featw['Word']=result1

In [ ]:
featw.head()

In [ ]:
data.head()

In [ ]:
model['MDN']

In [ ]:
type(model['MDN'])

In [ ]:
model['MDN']

In [ ]:
featw[featw['Word'] == 'CTW']

In [ ]:
for i in range(len(texts)):
    for k in range(len(texts[i])):
        if texts[i][k]=='CTW' :
            print("find CTW",'line: ',i,'column: ',k)

In [ ]:
len(featw.Word.unique())

In [ ]:
featw.shape

In [ ]:
vectors=pd.DataFrame([model[w]for w in (model.wv.vocab)])
vectors.columns=['vec_{}'.format(i) for i in range(256)]


In [ ]:
vectors.head()

In [ ]:
vectors['Word']=list(model.wv.vocab)

In [ ]:
vectors.head()

In [ ]:
model['VLP']

In [ ]:
vectors.head()

In [ ]:
data.head()

In [ ]:
feat.head()

In [ ]:
featw.head()

In [ ]:
featw=featw.merge(vectors,on='Word',how='left')

In [ ]:
featw.head()

In [ ]:
del featw['Word']

In [ ]:
featw=featw.groupby(['Protein_ID'],as_index="False").agg('mean')

In [ ]:
featw['Protein_ID']=featw.index

In [ ]:
featw.columns

In [ ]:
featw.head()

In [ ]:
data.head()

In [ ]:
data=data.merge(featw,on='Protein_ID',how='left')

In [ ]:
data.head()

In [ ]:
feat.head()

In [ ]:
feat=feat.astype('int')

In [ ]:
data=data.merge(feat,on='Molecule_ID',how='left')

In [ ]:
train_feat=data[data['Ki'] >-11].fillna(0)
test_feat=data[data['Ki'] <=-11].fillna(0)

In [ ]:
label_x=train_feat['Ki']
label_y=test_feat['Ki']

In [ ]:
train_feat.iloc[-5:]

In [ ]:
train_feat.drop(['Protein_ID','Molecule_ID','Ki'],axis=1,inplace=True)
test_feat.drop(['Protein_ID','Molecule_ID','Ki'],axis=1,inplace=True)

In [ ]:
train_feat.iloc[-5:]

In [ ]:
import lightgbm as lgb

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'l2',
    #'objective': 'multiclass',
    #'metric': 'multi_error',
    #'num_class':5,
    'min_child_weight': 3,
    'num_leaves': 2 ** 5,
    'lambda_l2': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'colsample_bylevel': 0.7,
    'learning_rate': 0.05,
    'tree_method': 'exact',
    'seed': 2017,
    'nthread': 4,
    'silent': True
    }

In [ ]:
import time

In [ ]:
train=lgb.Dataset(train_feat,label_x)
test=lgb.Dataset(test_feat,label=label_y,reference=train)

In [ ]:
train_feat.dtypes

In [ ]:
%%time
n=3000
gbm=lgb.train(params,
             train,
             n,
             verbose_eval=50,
             valid_sets=[train,test])

In [ ]:
pred_train=gbm.predict(train_feat)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
np.sqrt(mean_squared_error(label_x,pred_train))